In [11]:
import torchvision as tv
import pandas as pd
import numpy as np
import time 
import torch
from torch import nn

In [12]:
BATCH_SIZE = 512

In [13]:
train_dataset = tv.datasets.FashionMNIST('.', train=True, transform= tv.transforms.ToTensor(),download = True)
test_dataset = tv.datasets.FashionMNIST('.', train=False, transform= tv.transforms.ToTensor(),download = True)

In [14]:
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size = BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size = BATCH_SIZE)

In [5]:
train_dataset[0][0].shape

torch.Size([1, 28, 28])

In [37]:
x = 16*120
x

1920

In [44]:
model = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=4, padding=1),
    nn.ReLU(),
    nn.Conv2d(6, 16, kernel_size=4),
    nn.ReLU(),
    nn.AvgPool2d(2, stride=2),
    nn.Conv2d(16, 120, kernel_size=4, padding=1),
    nn.ReLU(),
    nn.AvgPool2d(2, stride=2),
    nn.Flatten(),
    nn.Linear(3000,512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Linear(256, 128),
    nn.BatchNorm1d(128),
    nn.ReLU(),
    nn.Linear(128,10)
)

In [8]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = torch.Tensor([0]), 0
    for X, y in data_iter:
        acc_sum += (net(X).argmax(axis = 1) == y).sum()
        n += y.shape[0]
    return acc_sum.item()/n

In [9]:
def train(net, train_iter, test_iter, optimizer, num_epochs):
    loss = nn.CrossEntropyLoss()
    
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0,0.0,0, time.time()
        
        for X, y in train_iter:
            optimizer.zero_grad()
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) ==y).sum().item()
            n += y.shape[0]
            
        test_acc = evaluate_accuracy(test_iter, net)
        print(f'epoch{epoch + 1}, loss {train_l_sum/ n:.4f}, train_acc{train_acc_sum/n:.3f}' \
              f', test_acc{test_acc:.3f}, time {time.time() - start:.1f} sec')

In [46]:
lr = 0.01
num_epochs = 6
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
train(model, train_iter, test_iter, optimizer, num_epochs)

epoch1, loss 0.0005, train_acc0.903, test_acc0.898, time 198.0 sec
epoch2, loss 0.0004, train_acc0.917, test_acc0.904, time 193.1 sec
epoch3, loss 0.0004, train_acc0.923, test_acc0.907, time 195.4 sec
epoch4, loss 0.0004, train_acc0.930, test_acc0.914, time 203.5 sec
epoch5, loss 0.0003, train_acc0.936, test_acc0.909, time 198.9 sec
epoch6, loss 0.0003, train_acc0.942, test_acc0.906, time 196.2 sec
